### OHLC DATA OF OPTIONS 
### 7:30 to 12:30 next day
### save this data to one put sheets and one call sheet then add them to main sheet 
### Column should have dates of expiry , row 1 should be time of closing and values to be added in them 


In [3]:
import requests
import pandas as pd
import datetime
from datetime import timezone



def convert_to_utc_unix(dt):
    """Converts a datetime object to UTC and then to Unix timestamp."""
    utc_dt = dt.astimezone(timezone.utc)
    unix_time = int(utc_dt.timestamp())
    return unix_time

# Assuming your start_time and end_time are already datetime objects
start_time = datetime.datetime(2024, 7, 1, 18, 30, 0)
end_time = datetime.datetime(2024, 7, 1, 20, 30, 0)

# Convert to UTC and then to Unix timestamp
unix_start = convert_to_utc_unix(start_time)
unix_end = convert_to_utc_unix(end_time)

print(unix_start)
print(unix_end)
symbol = "C-BTC-61800-020724"
resolution = "15m"
params = {
    'resolution': resolution,
    'symbol': symbol,
    'start': f"{unix_start}",
    'end': f"{unix_end}"
}
response = requests.get("https://cdn.india.deltaex.org/v2/history/candles", params=params)

# Extract the 'result' key which contains the data
data = response.json()['result']
close_data = [{'time': entry['time'], 'close': entry['close']} for entry in data]
print(data)
# Create a DataFrame
# df = pd.DataFrame(close_data)
# df['time'] = pd.to_datetime(df['time'], unit='s')
# df['time'] = df['time'].dt.tz_localize('UTC').dt.tz_convert('Asia/Kolkata')
# df['time'] = df['time'].dt.strftime('%H:%M:%S')

# Prepare the data for Excel
# time_values = df['time'].tolist()
# close_values = df['close'].tolist()

# # Create a new DataFrame for the Excel format
# df_excel = pd.DataFrame(columns=['EXPIRY DATE'] + time_values)
# df_excel.loc[1, 'EXPIRY DATE'] = end_time.strftime('%Y-%m-%d')
# for i, close in enumerate(close_values):
#     df_excel.loc[1, time_values[i]] = close

# # with pd.ExcelWriter('historical_data_o.xlsx', engine='xlsxwriter') as writer:
# #     df_excel.to_excel(writer, sheet_name='Calls', index=False)
# print(df)
print("Data saved to historical_data_o.xlsx")


1719838800
1719846000
[{'close': 1181, 'high': 1181, 'low': 1181, 'open': 1181, 'time': 1719845100, 'volume': 43}, {'close': 1046, 'high': 1046, 'low': 1046, 'open': 1046, 'time': 1719844200, 'volume': 0}, {'close': 1046, 'high': 1046, 'low': 1046, 'open': 1046, 'time': 1719843300, 'volume': 0}, {'close': 1046, 'high': 1046, 'low': 1046, 'open': 1046, 'time': 1719842400, 'volume': 0}, {'close': 1046, 'high': 1084, 'low': 1046, 'open': 1084, 'time': 1719841500, 'volume': 11}, {'close': 1109, 'high': 1109, 'low': 1109, 'open': 1109, 'time': 1719840600, 'volume': 0}, {'close': 1109, 'high': 1109, 'low': 1077, 'open': 1077, 'time': 1719839700, 'volume': 16}, {'close': 1201, 'high': 1201, 'low': 1201, 'open': 1201, 'time': 1719838800, 'volume': 10}]
Data saved to historical_data_o.xlsx


In [6]:
import requests
import math 
def get_historical_atm(unix_start,resolution,index):
        start = unix_start - 3600
        end = unix_start 
        resolution = resolution
        params = {
        'resolution': resolution,
        'symbol': f"{index}USD",
        'start': f"{start}",
        'end': f"{end}"
        }
        response = requests.get("https://cdn.india.deltaex.org/v2/history/candles", params=params)
        data = response.json()['result']
        close = data[1]['close']    
        atm_value =  int(math.floor(close / 200.0)) * 200
        return atm_value 

get_historical_atm(1719763200,'1h',"BTC")     

61600

In [6]:
import datetime
from datetime import timedelta
start_time = datetime.datetime(2024, 6, 30, 21, 30, 0)
end_time = start_time + datetime.timedelta(hours=12)
print(end_time.strftime('%d%m%y'))
utc_dt = "08/04/2022 12:00:00"



010724


AttributeError: module 'datetime' has no attribute 'strptime'

In [2]:
import datetime

def date(start_time_date):
    # Parse the input string into a datetime object
    date_obj = datetime.datetime.strptime(start_time_date, "%d-%m-%Y %I:%M %p")
    
    # Construct the start_time with the specified time
    start_time = datetime.datetime(date_obj.year, date_obj.month, date_obj.day, date_obj.hour, date_obj.minute, date_obj.second)
    
    # Calculate end_time as 12 hours after start_time
    end_time = start_time + datetime.timedelta(hours=12)
    
    return start_time, end_time

# Example usage
start, end = date("02-08-2024 9:30 PM")
print("Start Time:", start)
print("End Time:", end)


Start Time: 2024-08-02 21:30:00
End Time: 2024-08-03 09:30:00


### SAVING DATA TO EXCEL AFTER GETTING FROM MONGODB

In [1]:
import pymongo
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client['ohlc_data']  # replace with your database name
put_collection = db['P-BTC-3M-5-AM-15m']
call_collection = db['C-BTC-3M-5-AM-15m']

# Fetch data from MongoDB
put_data = list(put_collection.find({}))
call_data = list(call_collection.find({}))
print(put_data)
print(call_data)

# Helper function to transform MongoDB data to DataFrame
def transform_data(data):
    records = []
    for entry in data:
        expiry = entry['expiry']
        atm_value = entry['atm_value']
        for time, value in entry['data'].items():
            records.append({'expiry': expiry, 'time': time, 'value': value})
    return pd.DataFrame(records)

# Transform data
put_df = transform_data(put_data)
call_df = transform_data(call_data)

# Pivot the DataFrame
put_pivot_df = put_df.pivot(index='expiry', columns='time', values='value').reset_index()
call_pivot_df = call_df.pivot(index='expiry', columns='time', values='value').reset_index()

# Debug print statements to check column names
print("PUT DataFrame Columns:", put_pivot_df.columns)
print("CALL DataFrame Columns:", call_pivot_df.columns)

# Define the desired column order
desired_order = ['expiry',  "04:30:00", "04:45:00", "05:00:00", "05:15:00", "05:30:00",
    "05:45:00", "06:00:00", "06:15:00", "06:30:00", "06:45:00",
    "07:00:00", "07:15:00", "07:30:00", "07:45:00", "08:00:00",
    "08:15:00", "08:30:00", "08:45:00"]

# Ensure the DataFrame columns are in the desired order
put_pivot_df = put_pivot_df.reindex(columns=desired_order)
call_pivot_df = call_pivot_df.reindex(columns=desired_order)

# Debug print statements to check data after reindexing
print("PUT DataFrame after reindexing:")
print(put_pivot_df)
print("CALL DataFrame after reindexing:")
print(call_pivot_df)

# Calculate the total by summing put and call values
total_df = put_pivot_df.copy()
total_df.iloc[:, 1:] = put_pivot_df.iloc[:, 1:].fillna(0) + call_pivot_df.iloc[:, 1:].fillna(0)

# Debug print statements to check the total DataFrame
print("TOTAL DataFrame after summing PUT and CALL:")
print(total_df)

# Create an Excel workbook and worksheets using openpyxl
wb = Workbook()
put_ws = wb.active
put_ws.title = "Put"

call_ws = wb.create_sheet(title="Call")
total_ws = wb.create_sheet(title="Total")

# Write the pivot tables to the Excel sheets using openpyxl
for r_idx, row in enumerate(dataframe_to_rows(put_pivot_df, index=False, header=True)):
    for c_idx, value in enumerate(row):
        put_ws.cell(row=r_idx + 1, column=c_idx + 1, value=value)

for r_idx, row in enumerate(dataframe_to_rows(call_pivot_df, index=False, header=True)):
    for c_idx, value in enumerate(row):
        call_ws.cell(row=r_idx + 1, column=c_idx + 1, value=value)

for r_idx, row in enumerate(dataframe_to_rows(total_df, index=False, header=True)):
    for c_idx, value in enumerate(row):
        total_ws.cell(row=r_idx + 1, column=c_idx + 1, value=value)

# Save the Excel file
wb.save('btc-5am-7am-backtest.xlsx')

print("Data saved to .xlsx")


[{'_id': ObjectId('66afaea429cd4d66f76c15ea'), 'expiry': '01-04-24', 'atm_value': 71000, 'data': {'08:45:00': 430, '08:30:00': 388, '08:15:00': 371, '08:00:00': 371, '07:45:00': 384, '07:30:00': 384, '07:15:00': 384, '07:00:00': 343, '06:45:00': 338, '06:30:00': 313, '06:15:00': 330, '06:00:00': 389, '05:45:00': 311, '05:30:00': 311, '05:15:00': 311, '05:00:00': 421, '04:45:00': 421, '04:30:00': 421}}, {'_id': ObjectId('66afaea629cd4d66f76c15eb'), 'expiry': '02-04-24', 'atm_value': 69600, 'data': {'08:00:00': 2709, '07:45:00': 1219, '07:30:00': 513, '07:15:00': 375, '07:00:00': 375, '06:45:00': 375, '06:30:00': 375, '06:15:00': 639, '06:00:00': 639, '05:45:00': 401, '05:30:00': 360, '05:15:00': 238}}, {'_id': ObjectId('66afaea729cd4d66f76c15ec'), 'expiry': '03-04-24', 'atm_value': 65800, 'data': {'09:30:00': 164, '09:15:00': 180, '09:00:00': 244, '08:45:00': 244, '08:30:00': 397, '08:15:00': 488, '08:00:00': 488, '07:45:00': 488, '07:30:00': 739, '07:15:00': 739, '07:00:00': 739, '06:4